<a href="https://colab.research.google.com/github/gundaminpde/2022/blob/main/Transfer_L_VGG16_Children_Adults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###어린이와 어른의 사진을 구분하는 분류 모델을 VGG16을 통해 전이학습한다.

### 앞의 것과 다르게 kaggle에서 직접 자료를 받아 사용하였음.

### 자료는 https://www.kaggle.com/datasets/die9origephit/children-vs-adults-images?select=train 에서 받았음.

from google.colab import drive 
drive.mount('/content/drive/') 



Mounted at /content/drive/


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalMaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#trans_model = ResNet50(include_top=False,
#                 weights="imagenet",
#                 ##input_tensor=None,  ## include_top=False 이면 None
#                 input_shape=(200,200,3),    ## include_top=False 이면 None
#                 ##pooling=None,
#                 classes=1000) #imagenet을 하려면 1000을 값으로 씀

trans_model = VGG16(include_top=False, # VGG내부에 원래 있던 마지막 fully connected part를 사용할까? 말까?
                 weights="imagenet",
                 ##input_tensor=None,  ## include_top=False 이면 None
                 input_shape=(200,200,3),    ## include_top=False 이면 None
                 ##pooling=None,
                 classes=1000) #imagenet을 하려면 1000을 값으로 씀

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
trans_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [ ]:
trans_model.trainable = False

from tensorflow.keras.layers import Flatten, Dropout

model= Sequential()

model.add(trans_model)
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 10)                184330    
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                                 
Total params: 14,899,040
Trainable params: 184,352
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
pwd

'/content'

In [ ]:
cd /

/


In [ ]:
train_gen = ImageDataGenerator(
        rescale=1./255,         # 픽셀 값을 0~1 범위로 변환
        rotation_range=40,      # 40도까지 회전
        width_shift_range=0.2,  # 20%까지 좌우 이동
        height_shift_range=0.2, # 20%까지 상하 이동
        shear_range=0.2,        # 20%까지 기울임
        zoom_range=0.2,         # 20%까지 확대
        horizontal_flip=True,   # 좌우 뒤집기
    )


train = train_gen.flow_from_directory(directory='../content/drive/MyDrive/data/train',
                                              target_size=(200, 200),  # 사용할 CNN 모델 입력 사이즈에 맞게 resize
                                              color_mode='rgb',       # 컬러는 rgb, 흑백은 grayscale. 생략하면 컬러로 처리한다
                                              class_mode='categorical', # class_mode='binary',
                                              batch_size=20,
                                              shuffle=True)

valid_gen = ImageDataGenerator(
        rescale=1./255        
    ) 


valid = valid_gen.flow_from_directory(directory='../content/drive/MyDrive/data/test',
                                              target_size=(200, 200),  # 사용할 CNN 모델 입력 사이즈에 맞게 resize
                                              color_mode='rgb',       # 컬러는 rgb, 흑백은 grayscale. 생략하면 컬러로 처리한다
                                              class_mode='categorical', # class_mode='binary',
                                              batch_size=3,
                                              shuffle=True)

hist = model.fit(train, epochs=3, validation_data=valid, verbose=1)



Found 680 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/3
34/34 [==============================] - 301s 9s/step - loss: 0.7527 - accuracy: 0.4838 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/3
34/34 [==============================] - 296s 9s/step - loss: 0.6932 - accuracy: 0.4794 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/3
34/34 [==============================] - 295s 9s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
